In [1]:
from non_graphs_baselines import NonGraphBaseline

In [2]:
model_type = "timeseries"
mode = "long_term"
model = NonGraphBaseline(model_type, load_data=True)

In [7]:
model.train.__dict__.keys()

dict_keys(['edge_index', 'edge_weight', 'features', 'targets', 'additional_feature_keys', 'snapshot_count', 't'])

In [24]:
model.train.edge_index

array([[  0,   1,   2, ..., 324, 324, 324],
       [  0,   1,   2, ..., 257, 310, 324]])

In [11]:
model.train.edge_index.shape

(2, 2694)

In [18]:
model.train.snapshot_count

41665

In [21]:
print(len(model.train.features))
print(model.train.features[0].shape)

41665
(325, 2, 12)


In [1]:
print(len(model.train.targets))
print(model.train.targets[0].shape)

NameError: name 'model' is not defined

In [21]:
# Feature vector's second dimension (normalized time)
print(model.train.features[0][0][1])
print(model.train.features[-1][0][1])

AttributeError: 'function' object has no attribute 'features'

In [1]:
from agcrn import AGCRNWrapper

model = AGCRNWrapper(embed_dim=16)

In [2]:
print(model.model._gate.weights_pool.shape)

torch.Size([16, 3, 24, 24])


In [ ]:
import torch
optimizer = torch.optim.Adam(model.parameters(), lr=model.learning_rate)
optimizer.zero_grad()
model.train()
# subset = 10000
print("Running training...")
for epoch in range(20): 
    loss = 0
    step = 0
    for time, batch in enumerate(model.train_dataloader):
#         optimizer.zero_grad()
        out = model.model(batch.x, model.node_embeddings)
        loss = loss + model.loss_fn(out[:,0,:], batch.y[:,0,:])
        step += 1
    loss = loss / (step + 1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Epoch {} train MSE: {:.4f}".format(epoch, loss.item()))
    torch.save(model.state_dict(), "/Users/HarryShin/repos/CS224W_Traffic_Forecasting/agcrn_embed_16 "+str(epoch))
    model.eval()
    val_loss = 0
    step = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(model.test_dataloader):
            output = model.model(batch.x, model.node_embeddings)
            val_loss = val_loss + model.loss_fn(output[:,0,:], batch.y[:,0,:])
            step += 1
        val_loss = val_loss / step
        print("Epoch {} validation MSE: {:.4f}".format(epoch, val_loss.item()))
    model.train()


Running training...
Epoch 0 train MSE: 1.3497
Epoch 0 validation MSE: 1.0789
Epoch 1 train MSE: 1.0697
Epoch 1 validation MSE: 0.9010


In [1]:
from agcrn import AGCRNWrapper
import torch

model = AGCRNWrapper()
model.load_state_dict(torch.load("/Users/HarryShin/repos/CS224W_Traffic_Forecasting/checkpoints/agcrnfor15epochs 14"))


<All keys matched successfully>

In [2]:
pred, true = model.predict(None)

In [3]:
pred.shape

(125004, 325)

In [4]:
true.shape

(125004, 325)

In [3]:
from evaluate_model import ModelEvaluator

ev = ModelEvaluator(y_pred=pred, y_test=true, X_test=None, model_name="agcrn")

In [4]:
ev.evaluate()